In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words("russian"))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def summarize_text(text, compression_rate=0.1):
    # 1. Разбиваем текст на предложения
    sentences = sent_tokenize(text)
    num_sentences = len(sentences)
    
    # 2. Разбиваем текст на слова и нормализуем их (лемматизация)
    words = []
    for sentence in sentences:
        tokens = word_tokenize(sentence)
        tokens = [word.lower() for word in tokens if word.isalpha()]  # Только алфавитные токены
        words.extend(tokens)
    
    # 3. Удаляем стоп-слова
    filtered_words = [word for word in words if word not in stop_words]
    
    # 4. Подсчитываем веса слов (TF-IDF)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    word_scores = np.array(tfidf_matrix.sum(axis=0)).flatten()
    vocab = vectorizer.get_feature_names_out()
    word_weights = dict(zip(vocab, word_scores))
    
    # 5. Определяем веса предложений как сумму весов слов
    sentence_weights = []
    for sentence in sentences:
        sentence_weight = sum(word_weights.get(word.lower(), 0) for word in word_tokenize(sentence))
        sentence_weights.append(sentence_weight)
    
    # 6. Сортируем предложения по убыванию веса
    sentence_weight_pairs = list(enumerate(sentence_weights))
    sentence_weight_pairs.sort(key=lambda x: x[1], reverse=True)
    
    # 7. Оставляем предложения, входящие в заданный процент сжатия
    summary_sentence_count = max(1, int(num_sentences * compression_rate))
    selected_sentences = sorted([sentences[idx] for idx, _ in sentence_weight_pairs[:summary_sentence_count]])
    
    # 8. Формируем текст реферата
    summary_text = " ".join(selected_sentences)
    
    return {
        "keywords": sorted(word_weights.items(), key=lambda x: x[1], reverse=True)[:10],  # Топ-10 ключевых слов
        "sentence_weights": sentence_weight_pairs,
        "summary": summary_text
    }


In [3]:
# Пример текста
text = """
Москва — столица России, крупнейший город страны с многовековой историей и культурными достопримечательностями. 
В Москве находятся важные правительственные учреждения, включая Кремль и Государственную Думу. 
Город является также крупнейшим центром науки и образования, в нем расположены ведущие университеты и исследовательские институты. 
На протяжении веков Москва играла важную роль в экономике страны, и сегодня она остается финансовым центром России.
"""

# Установка коэффициента сжатия (например, 20% от оригинального текста)
compression_rate = 0.2

# Получение реферата
summary_data = summarize_text(text, compression_rate)

# Вывод ключевых слов, весов предложений и самого реферата
print("Ключевые слова с весами:", summary_data["keywords"])
print("Веса предложений:", summary_data["sentence_weights"])
print("Текст реферата:", summary_data["summary"])


Ключевые слова с весами: [('город', 0.4959630812544866), ('москва', 0.4853268010699042), ('россии', 0.4853268010699042), ('страны', 0.4853268010699042), ('центром', 0.44000993338245364), ('достопримечательностями', 0.34327249061384996), ('историей', 0.34327249061384996), ('крупнейший', 0.34327249061384996), ('культурными', 0.34327249061384996), ('многовековой', 0.34327249061384996)]
Веса предложений: [(3, 4.891323527630294), (2, 4.079704705810236), (0, 4.011578428147299), (1, 3.0)]
Текст реферата: На протяжении веков Москва играла важную роль в экономике страны, и сегодня она остается финансовым центром России.
